# Monthly Landsat DSWE Generator

This code employs Landsat 5, 7, 8, and 9 remote sensing data within the Dynamic Surface Water Extent (DSWE) algorithm to develop monthly water inundation extent maps and supplemental gridded products (QC raster and RGB of source composite) for a given study area. The code first creates monthly composites from avaialable Landsat data, then applies the algorithm. If the total study area has <95% coverage (accounting for cloud and cloud shadowed pixels), the non-classified areas are classified using a composite of Landsat imagery from +- 1 month. If the coverage is still < 95%, the remaining unclassifed area is classified using a range of +- 2 months and so on. The maximum expansion range default is +- 3 months, but may be edited. The resulting exports (GEE assets) include the DSWE product, the RGB bands of the source composite, and a QC product showing the total expansion range needed to classify each pixel, with 0 being the highest quality and 3 being the lowest.

DSWE Methodology: Jones, J.W., 2019. Improved Automated Detection of Subpixel-Scale Inundation—Revised Dynamic Surface Water Extent (DSWE) Partial Surface Water Tests. Remote Sensing 11, 374. https://doi.org/10.3390/rs11040374

Landsat Collection 2: Earth Resources Observation and Science (EROS) Center. (2020). Landsat 8-9 Operational Land Imager / Thermal Infrared Sensor Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9OGBGM6. Earth Resources Observation and Science (EROS) Center. (2020). Landsat 7 Enhanced Thematic Mapper Plus Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9C7I13B. Earth Resources Observation and Science (EROS) Center. (2020). Landsat 4-5 Thematic Mapper Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9IAXOVV.

Google Earth Engine: Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., Moore, R., 2017. Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote Sensing of Environment, Big Remotely Sensed Data: tools, applications and experiences 202, 18–27. https://doi.org/10.1016/j.rse.2017.06.031

Code adapted from Dr. Evan Greenberg: https://github.com/evan-greenbrg Greenberg, E., Chadwick, A.J., Ganti, V., 2023. A Generalized Area-Based Framework to Quantify River Mobility From Remotely Sensed Imagery. Journal of Geophysical Research: Earth Surface 128, e2023JF007189. https://doi.org/10.1029/2023JF007189

Author: James (Huck) Rees, PhD Student, UC Santa Barbara Geography

Date: August 18, 2025

## Import packages

In [1]:
import ee
import geopandas as gpd
import os
import logging
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks, argrelextrema
import matplotlib.pyplot as plt

ee.Initialize()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Initialize functions for generating monthly Landsat composites

In [2]:
def maskL8sr(image):
    """
    Masks out clouds and cloud shadows in Landsat 8/9 imagery using the BQA band.
    Uses bits 8–9 for cloud confidence and bits 10–11 for shadow confidence.

    Args:
        image: ee.Image, the input Landsat image.

    Returns:
        ee.Image: The masked image with cloud and shadow pixels removed.
    """
    qa = image.select('BQA')

    # Cloud confidence (bits 8–9): mask if medium or high
    cloud_conf = qa.rightShift(8).bitwiseAnd(3)
    cloud_ok = cloud_conf.lte(1)

    # Shadow confidence (bits 10–11): mask if medium or high
    shadow_conf = qa.rightShift(10).bitwiseAnd(3)
    shadow_ok = shadow_conf.lte(1)

    mask = cloud_ok.And(shadow_ok)
    return image.updateMask(mask)

def getLandsatCollection():
    """
    Merges Landsat 5, 7, 8, and 9 collections (Tier 1, Collection 2 SR) 
    and standardizes the band names for consistent analysis.

    Returns:
        ee.ImageCollection: A merged collection of standardized Landsat images.
    """
    # Define the band mappings for each Landsat version
    bn9 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
    bn8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
    bn7 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    bn5 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    # Standardized names for all bands
    standard_bands = ['uBlue', 'Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'BQA']

    # Fetch and rename bands in the Landsat collections
    ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").select(bn5, standard_bands)
    ls7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterDate('1999-04-15', '2003-05-30').select(bn7, standard_bands)
    ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(bn8, standard_bands)
    ls9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").select(bn9, standard_bands)

    # Merge all collections
    merged_collection = ls5.merge(ls7).merge(ls8).merge(ls9)

    return merged_collection

def rescale(image):
    """
    Rescale the reflectance values of Landsat imagery to allow for use of Landsat Collection 2 in DSWE.

    Parameters:
    image (ee.Image): The input image with bands to be rescaled.

    Returns:
    ee.Image: The image with rescaled bands added.
    """
    bns = ['uBlue', 'Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2']
    optical_bands = image.select(bns).multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, None, True)

def load_roi(shapefile_path):
    """Load ROI from a shapefile and return as an EE Geometry."""
    gdf = gpd.read_file(shapefile_path)
    return ee.Geometry.Polygon(gdf.unary_union.__geo_interface__["coordinates"])

def get_filled_composite_before_dswe(start_date, end_date, roi, max_months=6, fill_threshold=0.95):
    def get_composite(s, e):
        collection = (getLandsatCollection()
                      .map(maskL8sr)
                      .map(rescale)
                      .filterDate(s, e)
                      .filterBounds(roi))

        ids = collection.aggregate_array('system:index').distinct().sort()
        id_string = ids.join(",")  # Convert to comma-separated string
        return collection.median().clip(roi).set({'source_ids': id_string})

    def get_coverage(image):
        valid = image.select("Green").mask()
        stats = valid.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=30,
            maxPixels=1e13
        )
        return ee.Number(stats.get("Green"))

    base = get_composite(start_date, end_date)
    last_used_composite = base

    filled = base
    expansion_mask = base.select("Green").mask().Not().multiply(0).rename("expansion_mask")
    coverage = get_coverage(filled)

    for offset in range(1, max_months + 1):
        if coverage.gte(fill_threshold).getInfo():
            break

        delta = relativedelta(months=offset)
        expanded_start = (datetime.strptime(start_date, "%Y-%m-%d") - delta).strftime("%Y-%m-%d")
        expanded_end = (datetime.strptime(end_date, "%Y-%m-%d") + delta).strftime("%Y-%m-%d")
        extra = get_composite(expanded_start, expanded_end)
        last_used_composite = extra

        update_mask = base.select("Green").mask().Not()
        new_expansion = extra.select("Green").mask().And(update_mask).multiply(offset).rename("expansion_mask")
        expansion_mask = expansion_mask.where(expansion_mask.eq(0).And(new_expansion.neq(0)), new_expansion)

        filled = extra.blend(filled)
        coverage = get_coverage(filled)

    # Attach source scene metadata
    filled = filled.set({'source_ids': last_used_composite.get('source_ids')})
    return filled, expansion_mask

## Initialize functions for water masking

In [3]:
# Normalized Difference Water Index (MNDWI)
def Mndwi(image):
    """
    Calculate the Modified Normalized Difference Water Index (MNDWI) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MNDWI band named 'mndwi'.
    """
    return image.normalizedDifference(['Green', 'Swir1']).rename('mndwi')

# Modified Bare Soil Reflectance Variables
def Mbsrv(image):
    """
    Calculate the Modified Bare Soil Reflectance Variable (MBSRV) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MBSRV band named 'mbsrv'.
    """
    return image.select(['Green']).add(image.select(['Red'])).rename('mbsrv')

def Mbsrn(image):
    """
    Calculate the Modified Bare Soil Reflectance Normalized (MBSRN) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MBSRN band named 'mbsrn'.
    """
    return image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn')

# Normalized Difference Vegetation Index (NDVI)
def Ndvi(image):
    """
    Calculate the Normalized Difference Vegetation Index (NDVI) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the NDVI band named 'ndvi'.
    """
    return image.normalizedDifference(['Nir', 'Red']).rename('ndvi')

# Automated Water Extraction Index (AWESH)
def Awesh(image):
    """
    Calculate the Automated Water Extraction Index (AWEsh) for a given image.

    Parameters:
    image (ee.Image): The input image with the necessary bands for MBSRN calculation.

    Returns:
    ee.Image: The resulting image with the AWEsh band named 'awesh'.
    """
    return image.expression(
        'Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2',
        {
            'Blue': image.select(['Blue']),
            'Green': image.select(['Green']),
            'mbsrn': Mbsrn(image).select(['mbsrn']),
            'Swir2': image.select(['Swir2'])
        }
    ).rename('awesh')

def find_bimodal_trough(histogram_data, band_name='SWIR1', smoothing_sigma=2):
    """
    Find the trough (local minimum) between two peaks in a bimodal distribution.
    
    This implements the concept from Inman & Lyons (2020) of finding the natural 
    boundary between wet and dry pixels in the SWIR reflectance histogram. When 
    you plot SWIR values for a wetland image, you typically see two "humps" 
    (peaks): one for water/wet areas (low reflectance) and one for dry land 
    (high reflectance). The valley (trough) between these peaks represents the 
    natural separation point.
    
    Parameters:
    -----------
    histogram_data : dict
        GEE histogram output with structure: 
        {'BandName': {'bucketMeans': [...], 'histogram': [...]}}
    band_name : str
        Name of the band ('SWIR1' or 'SWIR2')
    smoothing_sigma : float
        Gaussian smoothing parameter to reduce noise in the histogram
        Higher values = smoother curve but may miss subtle features
        
    Returns:
    --------
    float : The reflectance value at the trough (threshold)
    dict : Diagnostic information about the distribution
    """
    
    # Extract histogram data
    band_data = histogram_data[band_name]
    means = np.array(band_data['bucketMeans'])  # Reflectance values (x-axis)
    counts = np.array(band_data['histogram'])    # Pixel counts (y-axis)
    
    # Smooth the histogram to reduce noise
    # Think of this like drawing a smooth curve through scattered points
    counts_smooth = gaussian_filter1d(counts, sigma=smoothing_sigma)
    
    # Find peaks (the two "humps" in the histogram)
    # prominence ensures we only find significant peaks, not small bumps
    peaks, peak_properties = find_peaks(
        counts_smooth, 
        prominence=np.max(counts_smooth) * 0.1  # Peak must be at least 10% of tallest peak
    )
    
    # If we don't find two clear peaks, fall back to percentile method
    if len(peaks) < 2:
        # Use the 30th percentile as a conservative wet/dry boundary
        cumsum = np.cumsum(counts)
        total = cumsum[-1]
        threshold_idx = np.where(cumsum >= total * 0.30)[0][0]
        threshold = means[threshold_idx]
        
        diagnostics = {
            'method': 'percentile_fallback',
            'threshold': threshold,
            'peaks_found': len(peaks),
            'wet_mode': None,
            'dry_mode': None,
            'reason': 'Bimodal structure not clear - using 30th percentile'
        }
        
        return threshold, diagnostics
    
    # Sort peaks by reflectance value (left to right on histogram)
    peak_indices = peaks[np.argsort(means[peaks])]
    
    # The first peak (leftmost) = wet mode (low reflectance)
    # The second peak (rightmost) = dry mode (high reflectance)
    wet_peak_idx = peak_indices[0]
    dry_peak_idx = peak_indices[1] if len(peak_indices) > 1 else peak_indices[0]
    
    # Find the lowest point (trough) between the two peaks
    search_range = counts_smooth[wet_peak_idx:dry_peak_idx+1]
    local_minima = argrelextrema(search_range, np.less)[0]
    
    if len(local_minima) > 0:
        # Take the deepest minimum (lowest point in the valley)
        trough_local_idx = local_minima[np.argmin(search_range[local_minima])]
        trough_idx = wet_peak_idx + trough_local_idx
        threshold = means[trough_idx]
    else:
        # Fallback: use midpoint between the two peaks
        threshold = (means[wet_peak_idx] + means[dry_peak_idx]) / 2
    
    # Package diagnostic information
    diagnostics = {
        'method': 'bimodal_trough',
        'threshold': threshold,
        'wet_mode': means[wet_peak_idx],           # Reflectance of wet peak
        'wet_mode_count': int(counts[wet_peak_idx]),  # Height of wet peak
        'dry_mode': means[dry_peak_idx],           # Reflectance of dry peak
        'dry_mode_count': int(counts[dry_peak_idx]),  # Height of dry peak
        'peaks_found': len(peaks),
        'trough_position': (threshold - means[wet_peak_idx]) / (means[dry_peak_idx] - means[wet_peak_idx])  # 0-1 scale
    }
    
    return threshold, diagnostics

def calculate_dynamic_swir2_threshold(image, roi, min_swir2=0.04, max_swir2=0.15, 
                                       save_plot=True, output_dir=None, 
                                       year=None, month=None):
    """
    Calculate dynamic SWIR2 threshold for a given image based on its histogram.
    
    This function analyzes the distribution of SWIR2 reflectance values across
    your study area and finds the natural separation between wet and dry pixels.
    The threshold is scene-specific and adapts to seasonal flooding conditions.
    
    This simplified version returns only SWIR2 threshold for use in Test 6
    (vegetated inundation enhancement).
    
    Parameters:
    -----------
    image : ee.Image
        The Landsat composite image
    roi : ee.Geometry
        Region of interest (your study area boundary)
    min_swir2, max_swir2 : float
        Safety constraints on SWIR2 threshold
        Default range: 0.04 to 0.15 (4% to 15% reflectance)
    save_plot : bool, optional (default=True)
        Whether to save histogram plot with threshold
    output_dir : str, optional (default=None)
        Directory to save plots. If None, saves to current directory
    year : int, optional
        Year for plot filename
    month : int, optional
        Month for plot filename
        
    Returns:
    --------
    float : The calculated SWIR2 threshold value
    """
    
    # Extract SWIR2 band
    swir2 = image.select(['Swir2'])
    
    # Get histogram from Google Earth Engine
    hist_dict = swir2.reduceRegion(
        reducer=ee.Reducer.histogram(maxBuckets=100),
        geometry=roi,
        scale=30,
        maxPixels=1e13
    ).getInfo()
    
    # Prepare histogram data for analysis
    swir2_hist = {'Swir2': hist_dict['Swir2']}
    
    # Find the trough (natural boundary) in histogram
    swir2_threshold, swir2_diag = find_bimodal_trough(swir2_hist, 'Swir2')
    
    # Apply safety constraints to prevent unreasonable values
    swir2_threshold_clipped = np.clip(swir2_threshold, min_swir2, max_swir2)
    
    # Create plot if requested
    if save_plot:
        import matplotlib.pyplot as plt
        import os
        
        # Extract histogram data
        means = np.array(hist_dict['Swir2']['bucketMeans'])
        counts = np.array(hist_dict['Swir2']['histogram'])
        
        # Create figure
        plt.figure(figsize=(10, 6))
        plt.bar(means, counts, width=(means[1] - means[0]) * 0.8, 
                color='steelblue', alpha=0.7, edgecolor='black', linewidth=0.5)
        
        # Add threshold line
        plt.axvline(swir2_threshold_clipped, color='red', linestyle='--', 
                   linewidth=2, label=f'Threshold = {swir2_threshold_clipped:.4f}')
        
        # Add wet and dry mode lines if available
        if swir2_diag.get('wet_mode') is not None:
            plt.axvline(swir2_diag['wet_mode'], color='blue', linestyle=':', 
                       linewidth=1.5, alpha=0.7, label=f'Wet Mode = {swir2_diag["wet_mode"]:.4f}')
        if swir2_diag.get('dry_mode') is not None:
            plt.axvline(swir2_diag['dry_mode'], color='orange', linestyle=':', 
                       linewidth=1.5, alpha=0.7, label=f'Dry Mode = {swir2_diag["dry_mode"]:.4f}')
        
        # Labels and title
        plt.xlabel('SWIR2 Reflectance', fontsize=12, fontweight='bold')
        plt.ylabel('Pixel Count', fontsize=12, fontweight='bold')
        
        if year and month:
            plt.title(f'SWIR2 Histogram with Dynamic Threshold\n{year}-{month:02d}', 
                     fontsize=14, fontweight='bold')
        else:
            plt.title('SWIR2 Histogram with Dynamic Threshold', 
                     fontsize=14, fontweight='bold')
        
        plt.legend(loc='upper right', fontsize=10)
        plt.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        
        # Determine output directory
        if output_dir is None:
            output_dir = os.getcwd()
        else:
            os.makedirs(output_dir, exist_ok=True)
        
        # Create filename
        if year and month:
            base_filename = f'SWIR2_threshold_{year}_{month:02d}'
        else:
            base_filename = 'SWIR2_threshold'
        
        # Save as PNG and JPEG
        png_path = os.path.join(output_dir, f'{base_filename}.png')
        jpeg_path = os.path.join(output_dir, f'{base_filename}.jpeg')
        
        plt.savefig(png_path, dpi=300, bbox_inches='tight')
        plt.savefig(jpeg_path, dpi=300, bbox_inches='tight', format='jpeg')
        plt.close()
        
        print(f"Plots saved:")
        print(f"  PNG:  {png_path}")
        print(f"  JPEG: {jpeg_path}")
    
    return float(swir2_threshold_clipped)

# Decision Tree for Surface Water Extent (DSWE)
def Dswe(image):
    """
    Calculate the Decision Tree for Surface Water Extent (DSWE) for a given image.

    Parameters:
    image (ee.Image): The input image with bands required for the DSWE calculation.

    Returns:
    ee.Image: The resulting image with the DSWE classification named 'dswe'.
    """
    mndwi = Mndwi(image)
    mbsrv = Mbsrv(image)
    mbsrn = Mbsrn(image)
    awesh = Awesh(image)
    swir1 = image.select(['Swir1'])
    nir = image.select(['Nir'])
    ndvi = Ndvi(image)
    blue = image.select(['Blue'])
    swir2 = image.select(['Swir2'])

    # Decision tree thresholds
    t1 = mndwi.gt(0.124)
    t2 = mbsrv.gt(mbsrn)
    t3 = awesh.gt(0)
    t4 = (mndwi.gt(-0.44)
          .And(swir1.lt(0.09))
          .And(nir.lt(0.15))
          .And(ndvi.lt(0.7)))
    t5 = (mndwi.gt(-0.5)
          .And(blue.lt(0.1))
          .And(swir1.lt(0.3))
          .And(swir2.lt(0.1))
          .And(nir.lt(0.25)))

    # Combine results using weights to create unique classes
    t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))

    # Define DSWE classification levels
    noWater = t.eq(0).Or(t.eq(1)).Or(t.eq(10)).Or(t.eq(100)).Or(t.eq(1000))
    hWater = t.eq(1111).Or(t.eq(10111)).Or(t.eq(11011)).Or(t.eq(11101)).Or(t.eq(11110)).Or(t.eq(11111))
    mWater = (t.eq(111).Or(t.eq(1011)).Or(t.eq(1101)).Or(t.eq(1110))
              .Or(t.eq(10011)).Or(t.eq(10101)).Or(t.eq(10110))
              .Or(t.eq(11001)).Or(t.eq(11010)).Or(t.eq(11100)))
    pWetland = t.eq(11000)
    lWater = (t.eq(11).Or(t.eq(101)).Or(t.eq(110)).Or(t.eq(1001))
              .Or(t.eq(1010)).Or(t.eq(1100)).Or(t.eq(10000))
              .Or(t.eq(10001)).Or(t.eq(10010)).Or(t.eq(10100)))

    # Assign classification levels to DSWE
    iDswe = (noWater.multiply(0)
             .add(hWater.multiply(4))
             .add(mWater.multiply(3))
             .add(pWetland.multiply(2))
             .add(lWater.multiply(1)))

    return iDswe.rename(['dswe'])

def Dswe_with_Test6(image, roi, min_swir2=0.04, max_swir2=0.15, 
                     save_plot=True, output_dir=None, year=None, month=None):
    """
    Calculate DSWE classification with Test 6 enhancement for vegetated inundation.
    
    This function applies the standard DSWE algorithm, then upgrades class confidence
    for pixels that also pass Test 6 (SWIR2 < dynamic threshold). This enhancement
    is designed to better capture water beneath dense vegetation (e.g., papyrus swamps)
    where traditional spectral indices may fail but SWIR2 still indicates moisture.
    
    Upgrade logic:
    - Class 0 (No Water) + Test 6 pass → Class 1 (Low Water)
    - Class 1 (Low Water) + Test 6 pass → Class 2 (Partial Wetland)
    - Class 2 (Partial Wetland) + Test 6 pass → Class 3 (Moderate Water)
    - Class 3 (Moderate Water) + Test 6 pass → Class 4 (High Water)
    - Class 4 (High Water) → Remains Class 4 (no change)
    
    Parameters:
    -----------
    image : ee.Image
        Landsat composite with standard bands
    roi : ee.Geometry
        Region of interest for threshold calculation
    min_swir2, max_swir2 : float
        Safety constraints on SWIR2 threshold
        Default range: 0.04 to 0.15 (4% to 15% reflectance)
    save_plot : bool, optional (default=True)
        Whether to save SWIR2 histogram plot
    output_dir : str, optional (default=None)
        Directory to save plots
    year : int, optional
        Year for metadata and plot filename
    month : int, optional
        Month for metadata and plot filename
        
    Returns:
    --------
    tuple : (upgraded_classification, original_classification, swir2_threshold)
        - upgraded_classification: ee.Image with Test 6 upgrades applied
        - original_classification: ee.Image with standard DSWE (for comparison)
        - swir2_threshold: float, the calculated SWIR2 threshold value
    """
    
    # Step 1: Run standard DSWE algorithm
    original_dswe = Dswe(image)
    
    # Step 2: Calculate dynamic SWIR2 threshold
    swir2_threshold = calculate_dynamic_swir2_threshold(
        image, roi, 
        min_swir2=min_swir2, 
        max_swir2=max_swir2,
        save_plot=save_plot,
        output_dir=output_dir,
        year=year,
        month=month
    )
    
    # Step 3: Create Test 6 (SWIR2 < threshold)
    swir2 = image.select(['Swir2'])
    test6 = swir2.lt(swir2_threshold)
    
    # Step 4: Apply upgrade logic
    # Start with original classification
    upgraded_dswe = original_dswe
    
    # Upgrade class 0 → 1 if Test 6 passes
    upgraded_dswe = upgraded_dswe.where(
        original_dswe.eq(0).And(test6), 
        1
    )
    
    # Upgrade class 1 → 2 if Test 6 passes
    upgraded_dswe = upgraded_dswe.where(
        original_dswe.eq(1).And(test6), 
        2
    )
    
    # Upgrade class 2 → 3 if Test 6 passes
    upgraded_dswe = upgraded_dswe.where(
        original_dswe.eq(2).And(test6), 
        3
    )
    
    # Upgrade class 3 → 4 if Test 6 passes
    upgraded_dswe = upgraded_dswe.where(
        original_dswe.eq(3).And(test6), 
        4
    )
    
    # Class 4 remains unchanged (no .where() operation needed)
    
    # Step 5: Add metadata to both images
    metadata = {
        'swir2_threshold': swir2_threshold,
        'test6_applied': True,
        'algorithm': 'DSWE_with_Test6_vegetated_enhancement'
    }
    
    if year is not None:
        metadata['year'] = year
    if month is not None:
        metadata['month'] = month
    
    upgraded_dswe = upgraded_dswe.set(metadata).rename(['dswe'])
    original_dswe = original_dswe.set({
        'swir2_threshold': swir2_threshold,
        'test6_applied': False,
        'algorithm': 'DSWE_standard'
    }).rename(['dswe'])
    
    return upgraded_dswe, original_dswe, swir2_threshold

# Function to generate a water mask for a given year and polygon feature
def get_water_mask_for_feature(start_date, end_date, polygon, 
                                min_swir2=0.04, max_swir2=0.15,
                                save_plot=True, output_dir=None, 
                                year=None, month=None,
                                return_original=False):
    """
    Generate a water mask for a given date range and polygon feature using Landsat imagery
    with DSWE Test 6 enhancement for vegetated inundation.
    
    Parameters:
    -----------
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format
    polygon : ee.Geometry.Polygon
        The polygon feature defining the area of interest
    min_swir2, max_swir2 : float, optional
        Safety constraints on SWIR2 threshold (default: 0.04 to 0.15)
    save_plot : bool, optional (default=True)
        Whether to save SWIR2 histogram plot
    output_dir : str, optional (default=None)
        Directory to save plots
    year : int, optional
        Year for metadata and plot filename
    month : int, optional
        Month for metadata and plot filename
    return_original : bool, optional (default=False)
        If True, returns tuple of (upgraded_mask, original_mask, threshold)
        If False, returns only upgraded_mask
    
    Returns:
    --------
    ee.Image or tuple : 
        If return_original=False: upgraded water mask image
        If return_original=True: (upgraded_mask, original_mask, swir2_threshold)
    """
    
    # Build Landsat composite
    imagery = (getLandsatCollection()
               .map(maskL8sr)
               .map(rescale)
               .filterDate(start_date, end_date)
               .filterBounds(polygon))
    
    image_composite = imagery.median().clip(polygon)
    
    # Apply DSWE with Test 6
    upgraded_mask, original_mask, swir2_threshold = Dswe_with_Test6(
        image_composite,
        polygon,
        min_swir2=min_swir2,
        max_swir2=max_swir2,
        save_plot=save_plot,
        output_dir=output_dir,
        year=year,
        month=month
    )
    
    if return_original:
        return upgraded_mask, original_mask, swir2_threshold
    else:
        return upgraded_mask

def export_to_asset(image, year, month, roi, asset_folder, bands=None, product_name="DSWE"):
    """
    Export a single-band or multi-band Earth Engine image to a GEE asset, 
    with proper metadata and asset ID formatting.
    
    Parameters:
        image (ee.Image): The image to export
        year (int): The image year
        month (int): The image month
        roi (ee.Geometry): Region of interest for clipping/export
        asset_folder (str): GEE asset folder path (no trailing slash)
        bands (list of str): Specific bands to export
        product_name (str): Prefix for the image asset name (e.g. "DSWE", "QC", "Composite")
    """
    asset_id = f"{asset_folder}/{product_name}_{year}_{month:02d}"
    
    # Select only specified bands if provided
    if bands:
        image_to_export = image.select(bands)
    else:
        image_to_export = image

    # Compute last day of month
    last_day = calendar.monthrange(year, month)[1]

    # Prepare image metadata
    image_with_metadata = image_to_export.set({
        'system:time_start': ee.Date(f"{year}-{month:02d}-01").millis(),
        'system:time_end': ee.Date(f"{year}-{month:02d}-{last_day:02d}").millis(),
        'source_ids': image.get('source_ids')
    })

    # Check if the asset already exists
    try:
        ee.data.getAsset(asset_id)
        logging.info(f"Skipping {asset_id}, already exists.")
    except:
        # Export the image with metadata
        task = ee.batch.Export.image.toAsset(
            image=image_with_metadata,
            description=f"{product_name}_{year}_{month:02d}",
            assetId=asset_id,
            scale=30,
            region=roi,
            maxPixels=1e13
        )
        task.start()
        logging.info(f"Exporting {asset_id}...")

def process_monthly_dswe(start_date, end_date, shapefile_path, asset_folder,
                         export_dswe=True, export_original_dswe=False, 
                         export_qc=True, export_composite=True, export_swir2=True,
                         min_swir2=0.04, max_swir2=0.15,
                         save_swir2_plots=True, plot_output_dir=None):
    """
    Generate and export DSWE composites with Test 6 enhancement for each month within 
    the given date range, with optional export of DSWE, original DSWE, QC, RGB composite, 
    and SWIR2 products, each to its own fixed subfolder.
    
    Parameters:
    -----------
    start_date : datetime
        Start date for processing
    end_date : datetime
        End date for processing
    shapefile_path : str
        Path to the ROI shapefile
    asset_folder : str
        Base folder path for asset exports
    export_dswe : bool, optional (default=True)
        Whether to export DSWE water classification products (with Test 6 enhancement)
    export_original_dswe : bool, optional (default=False)
        Whether to export original DSWE classification (without Test 6) for comparison
    export_qc : bool, optional (default=True)
        Whether to export QC mask products
    export_composite : bool, optional (default=True)
        Whether to export RGB Landsat composite products
    export_swir2 : bool, optional (default=True)
        Whether to export SWIR2 band products
    min_swir2, max_swir2 : float, optional
        Safety constraints on SWIR2 threshold (default: 0.04 to 0.15)
    save_swir2_plots : bool, optional (default=True)
        Whether to save SWIR2 histogram plots with threshold
    plot_output_dir : str, optional (default=None)
        Directory to save SWIR2 plots. If None, saves to current directory
    """
    # Fixed subfolder names
    DSWE_FOLDER = "DSWE_Products"
    DSWE_ORIGINAL_FOLDER = "DSWE_Original_Products"
    COMPOSITE_FOLDER = "Source_LS_Composites"
    QC_FOLDER = "QC_Masks"
    SWIR2_FOLDER = "SWIR_2_LS"
    
    roi = load_roi(shapefile_path)
    current_date = start_date
    
    while current_date <= end_date:
        year, month = current_date.year, current_date.month
        last_day = calendar.monthrange(year, month)[1]
        im_start_date = f"{year}-{month:02d}-01"
        im_end_date = f"{year}-{month:02d}-{last_day:02d}"
        
        try:
            # Get filled composite
            filled_composite, expansion_mask = get_filled_composite_before_dswe(
                im_start_date, im_end_date, roi
            )
            
            # Apply DSWE with Test 6
            logging.info(f"Processing DSWE with Test 6 for {year}-{month:02d}...")
            dswe_upgraded, dswe_original, swir2_threshold = Dswe_with_Test6(
                filled_composite,
                roi,
                min_swir2=min_swir2,
                max_swir2=max_swir2,
                save_plot=save_swir2_plots,
                output_dir=plot_output_dir,
                year=year,
                month=month
            )
            
            logging.info(f"  SWIR2 threshold: {swir2_threshold:.4f}")
            
            # Check for non-empty result
            if hasattr(dswe_upgraded, 'size') and dswe_upgraded.size().getInfo() == 0:
                logging.warning(f"No data available for {year}-{month:02d}.")
            else:
                # Export upgraded DSWE (with Test 6)
                if export_dswe:
                    export_to_asset(
                        dswe_upgraded,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{DSWE_FOLDER}",
                        bands=["dswe"],
                        product_name="DSWE"
                    )
                
                # Export original DSWE (without Test 6) for comparison
                if export_original_dswe:
                    export_to_asset(
                        dswe_original,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{DSWE_ORIGINAL_FOLDER}",
                        bands=["dswe"],
                        product_name="DSWE_Original"
                    )
                
                # Export QC raster
                if export_qc:
                    export_to_asset(
                        expansion_mask,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{QC_FOLDER}",
                        bands=["expansion_mask"],
                        product_name="QC"
                    )
                
                # Export RGB Landsat composite
                if export_composite:
                    export_to_asset(
                        filled_composite,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{COMPOSITE_FOLDER}",
                        bands=["Blue", "Green", "Red"],
                        product_name="Composite"
                    )
                
                # Export SWIR2 band
                if export_swir2:
                    export_to_asset(
                        filled_composite,
                        year,
                        month,
                        roi,
                        f"{asset_folder}/{SWIR2_FOLDER}",
                        bands=["Swir2"],
                        product_name="SWIR2"
                    )
                    
        except Exception as e:
            logging.warning(f"Failed to process {year}-{month:02d}: {e}")
        
        # Move to first of the next month
        current_date = (current_date.replace(day=28) + timedelta(days=4)).replace(day=1)

# Input parameters and export masks as GEE assets

In [5]:
# User-defined parameters
start_date = datetime(2015, 1, 1)
end_date = datetime(2017, 12, 31)
study_area_path = r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\StudyAreas\Delta_UCB\Delta_UCB_WGS84.shp"
gee_asset_output_folder = "projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3"

# Export options
export_dswe = True                    # Export DSWE with Test 6 enhancement
export_original_dswe = False          # Export original DSWE (without Test 6) for comparison
export_qc = False
export_composite = False
export_swir2 = False

# Test 6 parameters
min_swir2 = 0.04                      # Minimum allowed SWIR2 threshold
max_swir2 = 0.15                      # Maximum allowed SWIR2 threshold
save_swir2_plots = True               # Save histogram plots with thresholds
plot_output_dir = r"D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms"

# Process
process_monthly_dswe(
    start_date, 
    end_date, 
    study_area_path,  
    gee_asset_output_folder,
    export_dswe=export_dswe,
    export_original_dswe=export_original_dswe,
    export_qc=export_qc,
    export_composite=export_composite,
    export_swir2=export_swir2,
    min_swir2=min_swir2,
    max_swir2=max_swir2,
    save_swir2_plots=save_swir2_plots,
    plot_output_dir=plot_output_dir
)

2025-11-12 09:17:36,388 - INFO - Processing DSWE with Test 6 for 2015-01...
2025-11-12 09:17:37,863 - INFO -   SWIR2 threshold: 0.1329


Plots saved:
  PNG:  D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms\SWIR2_threshold_2015_01.png
  JPEG: D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms\SWIR2_threshold_2015_01.jpeg


2025-11-12 09:17:38,391 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/DSWE_Products/DSWE_2015_01...
2025-11-12 09:17:38,693 - INFO - Processing DSWE with Test 6 for 2015-02...
2025-11-12 09:17:40,319 - INFO -   SWIR2 threshold: 0.1368


Plots saved:
  PNG:  D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms\SWIR2_threshold_2015_02.png
  JPEG: D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms\SWIR2_threshold_2015_02.jpeg


2025-11-12 09:17:40,765 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/DSWE_Products/DSWE_2015_02...
2025-11-12 09:17:41,267 - INFO - Processing DSWE with Test 6 for 2015-03...
2025-11-12 09:17:43,076 - INFO -   SWIR2 threshold: 0.1487


...

2025-11-12 09:19:07,920 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/DSWE_Products/DSWE_2017_10...
2025-11-12 09:19:08,144 - INFO - Processing DSWE with Test 6 for 2017-11...
2025-11-12 09:19:09,786 - INFO -   SWIR2 threshold: 0.1500


Plots saved:
  PNG:  D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms\SWIR2_threshold_2017_11.png
  JPEG: D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms\SWIR2_threshold_2017_11.jpeg


2025-11-12 09:19:10,387 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/DSWE_Products/DSWE_2017_11...
2025-11-12 09:19:10,769 - INFO - Processing DSWE with Test 6 for 2017-12...
2025-11-12 09:19:12,615 - INFO -   SWIR2 threshold: 0.1500


Plots saved:
  PNG:  D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms\SWIR2_threshold_2017_12.png
  JPEG: D:\Okavango\Data\Water_Masks\Landsat\DSWE_w_SWIR2_threshold_test\SWIR2_histograms\SWIR2_threshold_2017_12.jpeg


2025-11-12 09:19:13,056 - INFO - Exporting projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v3/DSWE_Products/DSWE_2017_12...


## Monitor GEE tasks

In [ ]:
# Get list of all running GEE tasks
task_list = ee.batch.Task.list()

# Print task statuses
for task in task_list:
    print(f"Task: {task.status()['description']}, Status: {task.status()['state']}")
